In [ ]:
import os
import sys
import pandas as pd
import math
import matplotlib.pyplot as plt
import sweetviz as sv
import numpy as np
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, ParameterGrid, KFold
from sklearn.metrics import roc_auc_score, roc_curve, r2_score
from scipy.stats import spearmanr, pearsonr

In [ ]:
os.chdir('./scripts')

import load_config

In [ ]:
config = load_config.load_config_const()
plt.style.use('seaborn')

# Obtención de serie temporal de traspasos diarios de turismos

El objetivo de este notebook es crear una serie temporal en la que se calcule el número de traspasos de turismos. Primero realizaremos la carga del dataframe con una función que ya nos dará muchos de los campos formateados del notebook anterior

In [ ]:
df_dgt = load_config.obtain_dgt_dataset(config, 0.2)
df_dgt.sample(10)

El siguiente paso es agrupar todas las transacciones por las fechas en las que se producen. Esto nos dará la cuantía diaria de trasacciones

In [ ]:
temporal_df_dgt = df_dgt.groupby(["FEC_TRAMITE"]).size().reset_index(name='count')

In [ ]:
temporal_df_dgt.sort_values('FEC_TRAMITE', ascending=True)

Crearemos un nuevo campo para la fecha, ya que el objetivo es usarla como indice para ordenar el dataframa

In [ ]:
temporal_df_dgt['date'] = temporal_df_dgt['FEC_TRAMITE']
temporal_df_dgt

In [ ]:
temporal_df_dgt = temporal_df_dgt.set_index('FEC_TRAMITE')

In [ ]:
temporal_df_dgt

Mediante la siguiente linea de código, obtendremos las fechas en las cuales no tenemos registros de trasacciones (lo que interrumpiría la serie temporal)

In [ ]:
idx = pd.date_range(start = temporal_df_dgt["date"].min(), end = temporal_df_dgt["date"].max() , freq="d").difference(temporal_df_dgt["date"])

In [ ]:
idx

In [ ]:
temporal_df_dgt_no_data = pd.DataFrame({'FEC_TRAMITE': idx})
temporal_df_dgt_no_data

A partir de estas fechas obtendremos un nuevo dataframe, el cual iremos rellenando con las columnas de nuestro dataframe original

In [ ]:
temporal_df_dgt_no_data['date'] = temporal_df_dgt_no_data['FEC_TRAMITE']
temporal_df_dgt_no_data

In [ ]:
temporal_df_dgt_no_data['count'] = 0
temporal_df_dgt_no_data

Ya que en principio no tenemos datos de las ventas, indicamos que han sido 0, luego les daremos un valor

In [ ]:
temporal_df_dgt_no_data = temporal_df_dgt_no_data.set_index('FEC_TRAMITE')
temporal_df_dgt_no_data

Luego te tener nuestro nuevo dataframe con las fechas sin datos, lo concatenaremos a nuestro dataset original

In [ ]:
frames = [temporal_df_dgt, temporal_df_dgt_no_data]

result_dgt = pd.concat(frames)

In [ ]:
result_dgt.sort_values('date', ascending=False)

Indicaremos que días son fin de semana frente a los días ordinarios

In [ ]:
def is_weekend(day_week):
    if (day_week == 'Saturday' or day_week == 'Sunday'):
        return 1.0
    else:
        return 0.0
    return False

In [ ]:
result_dgt['is_weekend'] = result_dgt['date'].dt.day_name().apply(lambda x: is_weekend(x))

In [ ]:
result_dgt.sort_values('date', ascending=True)

Añadiremos nuevos campos para indicar los ciclos de semanas, meses y años. Tal como vimos en el notebook anterior, es importante recalcar estos periodos, ya que condicionan los traspasos y su frecuencia 

In [ ]:
x = np.arange(len(result_dgt))

result_dgt["s_period_week"] = np.sin(2*np.pi*x/7)
result_dgt["s_period_month"] = np.sin(2*np.pi*x/30.5)
result_dgt["s_period_year"] = np.sin(2*np.pi*x/365 )

result_dgt["c_period_week"] = np.cos(2*np.pi*x/7)
result_dgt["c_period_month"] = np.cos(2*np.pi*x/30.5)
result_dgt["c_period_year"] = np.cos(2*np.pi*x/365 )

In [ ]:
result_dgt.sort_values('date', ascending=True)

Llega el momento de rellenar los valores de los que no disponíamos información. Para ello diferenciaremos entre los días que son fin de semana de los que no, y con ello se les agregará dos medias diferentes

In [ ]:
result_dgt[(result_dgt['count'] == 0) & (result_dgt['is_weekend'] == 1.0)]['count']

In [ ]:
result_dgt[(result_dgt['count'] != 0) & (result_dgt['is_weekend'] == 1.0)]['count'].mean()

In [ ]:
result_dgt[(result_dgt['count'] == 0) & (result_dgt['is_weekend'] == 0.0)]['count']

In [ ]:
result_dgt[(result_dgt['count'] != 0) & (result_dgt['is_weekend'] == 0.0)]['count'].mean()

In [ ]:
result_dgt.loc[(result_dgt['count'] == 0) & (result_dgt['is_weekend'] == 1.0),'count'] = result_dgt[(result_dgt['count'] != 0) & (result_dgt['is_weekend'] == 1.0)]['count'].mean()

In [ ]:
result_dgt.loc[(result_dgt['count'] == 0) & (result_dgt['is_weekend'] == 0.0),'count'] = result_dgt[(result_dgt['count'] != 0) & (result_dgt['is_weekend'] == 0.0)]['count'].mean()

In [ ]:
result_dgt[(result_dgt['count'] == 0) & (result_dgt['is_weekend'] == 1.0)]

In [ ]:
result_dgt[(result_dgt['count'] == 0) & (result_dgt['is_weekend'] == 0.0)]

In [ ]:
result_dgt[result_dgt['date'] == '2015-02-21']

Hecho esto, ya tenemos nuestra linea de tiempo completa

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

sns.lineplot(data=result_dgt, x='date', y='count', ax=ax)

Para comparar nuestras predicciones con los valores reales, vamos a tomar un periodo de tiempo concreto, los últimos 4 meses del año 2021 (mostraremos los 5 últimos)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

sns.lineplot(data=result_dgt[result_dgt['date'] >= '2021-08-01' ], x='date', y='count', ax=ax)

Definiremos los campos predictores como el campo a predecir, pero añadiremos la fecha en ambos para realizar el corte de los últimos 4 meses

In [ ]:
X = result_dgt[[
    'date', 'is_weekend', 's_period_week', 's_period_week', 's_period_year', 'c_period_week', 'c_period_week', 'c_period_year'
]]

y = result_dgt[['date','count']]

In [ ]:
X

Obtenemos los datos de entrenamiento y los de test (los últimos 4 meses)

In [ ]:
train_X = train_X[train_X['date'] < '2021-09-01']
test_X = test_X[test_X['date'] >= '2021-09-01']

train_y = train_y[train_y['date'] < '2021-09-01']
test_y = test_y[test_y['date'] >= '2021-09-01']

Y, luego de ello, eliminamos la columna de fecha

In [ ]:
X = X[['is_weekend', 's_period_week', 's_period_week', 's_period_year', 'c_period_week', 'c_period_week', 'c_period_year']]
y = y['count']

train_X = train_X[['is_weekend', 's_period_week', 's_period_week', 's_period_year', 'c_period_week', 'c_period_week', 'c_period_year']]
test_X = test_X[['is_weekend', 's_period_week', 's_period_week', 's_period_year', 'c_period_week', 'c_period_week', 'c_period_year']]

train_y = train_y['count']
test_y = test_y['count']

Realizaremos las predicciones usando un RandomForestRegressor. Primero realizaremos una predicción con unos metadatos al azar para ver como se comporta nuestro modelo

In [ ]:
rf = RandomForestRegressor(n_estimators=600, min_samples_split=5, oob_score=True, max_depth=5)
rf.fit(train_X, train_y)

In [ ]:
rf.oob_score_

In [ ]:
rf.predict(train_X)

In [ ]:
y_hat = rf.predict(test_X)

test_score = r2_score(test_y, y_hat)


print(f'Out-of-bag R-2 score estimate: {rf.oob_score_:>5.3}')
print(f'Test data R-2 score: {test_score:>5.3}')

In [ ]:
from sklearn.model_selection import cross_val_score

cross_val_score(X=train_X,y=train_y, estimator=rf )

LLega el turno de usar un GridSearchCV para comprobar que configuración de nuestro modelo tiene mejores predicciones

In [ ]:
grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 4, 6, 8, 10],
    'max_leaf_nodes': [5,10,25,50,100]
}


grid_search = GridSearchCV(
    RandomForestRegressor(),
    param_grid=grid,
    scoring="neg_mean_squared_error",
    verbose=2
)

In [ ]:
grid_search.fit(train_X,train_y)

In [ ]:
grid_search.best_params_

In [ ]:
pd.DataFrame(grid_search.cv_results_)

Con la configuración óptima, volvemos a crear un nuevo modelo para tomar mediciones

In [ ]:
rf_final = RandomForestRegressor(n_estimators=200, min_samples_split=2, oob_score=True, max_depth=5, max_leaf_nodes=5)
rf_final.fit(train_X, train_y)

In [ ]:
rf_final.oob_score_

Y hacemos una comparativa de resultados con el primer modelo

In [ ]:
y_hat = rf_final.predict(test_X)

test_score = r2_score(test_y, y_hat)


print(f'Out-of-bag R-2 score estimate: {rf_final.oob_score_:>5.3}')
print(f'Test data R-2 score: {test_score:>5.3}')

In [ ]:
cross_val_score(X=train_X,y=train_y, estimator=rf_final )

In [ ]:
y_hat = rf.predict(test_X)

test_score = r2_score(test_y, y_hat)


print(f'Out-of-bag R-2 score estimate: {rf.oob_score_:>5.3}')
print(f'Test data R-2 score: {test_score:>5.3}')

In [ ]:
cross_val_score(X=train_X,y=train_y, estimator=rf )

Si bien los resultados son mejores, esa mejoría es apenas perceptible en los resultados

Llega el momento de hacer una comparación entre nuestras predicciones y la realidad

In [ ]:
df_graph_final = test_X.copy()

In [ ]:
df_graph_final.sample(10)

In [ ]:
df_graph_final['date'] = df_graph_final.index

In [ ]:
df_graph_final.sample(10)

In [ ]:
df_graph_final['count'] = rf_final.predict(test_X)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

sns.lineplot(data=result_dgt[result_dgt['date'] >= '2021-08-01' ], x='date', y='count', ax=ax, color='blue', label='Datos reales')
sns.lineplot(data=df_graph_final, x='date', y='count', ax=ax, color='red', label='Predicción')

## Conclusiones

- El modelo consigue predecir el descenso de transacciones los fines de semana
- Dependiendo de las iteraciones, consigue o no el aumento al final de año (esto puede ser debido al fracionamiento de los datos, ya que a cada generación se trae un porcentaje bajo de los mismo, pero a un volumen mayor el kernel se satura)